In [ ]:
import re

import Bio.Restriction as Restriction
import pandas as pd
import pygsheets
import requests
import toml
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.api as api
import paulssonlab.api.geneious as geneious
import paulssonlab.cloning.enzyme as enzyme
import paulssonlab.cloning.registry as registry
import paulssonlab.cloning.sequence as sequence
import paulssonlab.cloning.workflow as workflow

# Setup

In [ ]:
config = toml.load("config.toml")

In [ ]:
gc = pygsheets.authorize(service_account_file="credentials.json")

In [ ]:
reg = registry.Registry(gc, config["registry"]["folder"])

# Config

In [ ]:
otx_oligos = reg[("oTX", "oligos")]
# ptx_plasmids = reg[("pTX", "plasmids")]
# ptx_maps = reg[("pTX", "maps")]
# ftx_fragments = reg[("fTX", "fragments")]
# part_types = reg[("fTX", "fragments", "Part types")]

# Initialize TX registry

In [ ]:
reg.duplicate_collection("LIB", "TX")

# Dark rcVenus primers

In [ ]:
primers_sheet = pd.read_csv("/Users/jacob/Downloads/final_rcVenus_primers.csv")

In [ ]:
primers_sheet.columns

In [ ]:
primers_sheet["forward_primer"].str.len()

In [ ]:
base = {"Author": "Jacob Quinn Shenker", "Date": workflow.date()}

oligo_base = {
    **base,
    "Order date": workflow.date(),
    "Vendor": "Genewiz",
    "Type": "Primer",
}

part_base = {
    **base,
    "Tags": part_tags,
    "Reference": reference,
}

for primer in primer_sheet.iterrows():
    name = f"rcvenus_{}"
    oligo_id = olib_oligos.upsert(
        {
            **oligo_base,
            **row,
            "Name": f"{name}_sense",
            "Sequence": workflow.normalize_seq_upper(seq),
        },
        apply=apply,
    )
    oligo_id2 = olib_oligos.upsert(
        {
            **oligo_base,
            **row,
            "Name": f"{name}_antisense",
            "Sequence": workflow.normalize_seq_upper(sequence.reverse_complement(seq)),
        },
        apply=apply,
    )
    usage = f"{oligo_id}={oligo_id2}/{part_enzyme}"
    part_seq = workflow.normalize_seq(workflow.re_digest_part(seq, part_enzyme))
    part_row = {
        **part_base,
        **row,
        "Sequence": part_seq,
        "Usage": usage,
        "Type": part_type,
        "Upstream overhang": part_overhangs[0],
        "Downstream overhang": part_overhangs[1],
        "Species/codon usage": "E. coli",
    }
    lib_parts[name] = part_row

In [ ]:
base = {"Author": "Jacob Quinn Shenker", "Date": workflow.date()}

oligo_base = {
    **base,
    "Order date": workflow.date(),
    "Vendor": "Genewiz",
    "Type": "Primer",
}

part_base = {
    **base,
    "Tags": part_tags,
    "Reference": reference,
}

# apply = {"Sequence": workflow.normalize_seq}
apply = {"Name": None}

for name, row in seqs.items():
    seq = row["Sequence"]
    oligo_id = olib_oligos.upsert(
        {
            **oligo_base,
            **row,
            "Name": f"{name}_sense",
            "Sequence": workflow.normalize_seq_upper(seq),
        },
        apply=apply,
    )
    oligo_id2 = olib_oligos.upsert(
        {
            **oligo_base,
            **row,
            "Name": f"{name}_antisense",
            "Sequence": workflow.normalize_seq_upper(sequence.reverse_complement(seq)),
        },
        apply=apply,
    )
    usage = f"{oligo_id}={oligo_id2}/{part_enzyme}"
    part_seq = workflow.normalize_seq(workflow.re_digest_part(seq, part_enzyme))
    part_row = {
        **part_base,
        **row,
        "Sequence": part_seq,
        "Usage": usage,
        "Type": part_type,
        "Upstream overhang": part_overhangs[0],
        "Downstream overhang": part_overhangs[1],
        "Species/codon usage": "E. coli",
    }
    lib_parts[name] = part_row